In [50]:
from openai import OpenAI
import os
import json

In [51]:
# Example dummy functions
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "temperature": "62", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def get_current_time(location):
    """Get the current time in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "time": "15:00"})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "time": "05:00"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "time": "11:00"})
    else:
        return json.dumps({"location": location, "time": "unknown"})

In [52]:
# Set up OpenAI API credentials
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

In [53]:
# Function to generate a response using OpenAI's language model
def generate_response(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. New York, NY",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. New York, NY",
                        },
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # check if the model wanted to call a function
    if tool_calls:
        # call the function
        available_functions = {
            "get_current_weather": get_current_weather,
            "get_current_time": get_current_time,
        }  # multiple functions available
        messages.append(response_message)  # extend conversation with assistant's reply
        # send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name == "get_current_weather":
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"),
                )
            elif function_name == "get_current_time":
                function_response = function_to_call(
                    location=function_args.get("location"),
                )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content

In [54]:
# Main function to handle user queries
def handle_query(query):
    response = generate_response(query)
    return response

In [55]:
queries = [
    "How's the weather in New York, and what time is it there?",
    "What time is it in Tokyo right now?",
    "What's the weather in Paris?"
]

for query in queries:
    response = handle_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}\n")

Query: How's the weather in New York, and what time is it there?
Response: Currently, in New York, the temperature is around 62°F. The local time there is 5:00 AM.

Query: What time is it in Tokyo right now?
Response: The current time in Tokyo, Japan is 3:00 PM.

Query: What's the weather in Paris?
Response: The current temperature in Paris is 22 degrees.

